In [1]:
!apt-get update -y
!apt-get install -y python-xlsxwriter
!pip install openpyxl

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Err:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,277 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates/main i386 Packages [1,84

In [9]:
import wandb
import pandas as pd
import datetime
import tqdm

username = "memmelma"
project = "final" # multi_modal

filter_name = "dino_act_rgb_freeze"
filter_type = "eval_"
filter_state = "finished"

metrics = ["eval_metricssuccess", "eval_metricsspl", "eval_metricssoftspl", "eval_metricsdistance_to_goal"]

configs = ["config.VO.REGRESS_MODEL.cls_action", "config.VO.REGRESS_MODEL.visual_backbone", "config.VO.REGRESS_MODEL.pretrain_backbone", "config.VO.REGRESS_MODEL.visual_type"]

out_name = "exp_eval_strip"

In [10]:
# retreive runs

run_ids = []
run_names = []
run_ckpts = []

api = wandb.Api(timeout=45)
runs = api.runs(f"{username}/{project}")

runs_df = pd.DataFrame(columns=["name", "id", "state", *configs, *metrics])

for run in tqdm.tqdm(runs):
    
    if filter_state and run._state != filter_state:
        continue
    
    if filter_type and filter_type not in run.name:
        continue
        
    if filter_name and filter_name not in run.name:
        continue
    
    run_dict = {}
    
    # add base attributes
    run_dict["name"] = run.name.split("[")[0]
    run_dict["id"] = run.id 
    run_dict["state"] = run._state
    
    # add config
    for config in configs:
        # traverse config keys
        run_config_tmp = run.config
        for config_iter in config.split('.'):
            run_config_tmp = run_config_tmp[config_iter]
        run_dict[config] = run_config_tmp
    
    # add filter_operation filter_metric
    run_history = run.history()
    
    # add other metrics corresponding to filter_operation filter_metric
    for metric in metrics:
        # catch models that only evaluate some actions
        if metric in run_history.keys():
            run_dict[metric] = run_history[metric][0]
        else:
            run_dict[metric] = -1
    
    runs_df = runs_df.append(run_dict, ignore_index=True)
    
runs_df = runs_df.sort_values(by=['name'])
runs_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:01<00:00, 83.28it/s]


,name,id,state,config.VO.REGRESS_MODEL.cls_action,config.VO.REGRESS_MODEL.visual_backbone,config.VO.REGRESS_MODEL.pretrain_backbone,config.VO.REGRESS_MODEL.visual_type,eval_metricssuccess,eval_metricsspl,eval_metricssoftspl,eval_metricsdistance_to_goal
0,eval_vit_b_dino_act_rgb_freeze,ldy30iih,finished,True,base,dino,"[rgb, top_down_view]",0.112676,0.084547,0.464224,2.408709


In [5]:
dt = datetime.datetime.now().strftime("%Y%m%d")
filename = f"{out_name}_{dt}"
filename

'exp_eval_mmmae_20220512'

In [6]:
# save df as excel file
extension = ".xlsx"
with pd.ExcelWriter(filename+extension) as excel_writer:
    runs_df.to_excel(excel_writer, sheet_name='Sheet1')